<a href="https://www.kaggle.com/code/aditya9790/soc-estimation-lstm?scriptVersionId=126402860" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
import time, random
from datetime import datetime, timedelta
import pytz, plotly
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import os

BASE_PATH = '/kaggle/input/lg-18650hg2-liion-battery-data/Dataset_Li-ion'
print(os.listdir(os.path.join(BASE_PATH, 'LG_HG2_Original_Dataset_McMasterUniversity_Jan_2020')))

['Plot_results.m', 'Readme file - Description of Experimental Tests.txt', '10degC', '0degC', 'n10degC', 'n20degC', '25degC', '40degC', 'Technical Information and Experimental Test Results for LG 18650HG2.pdf']


In [2]:
!pip install nb_black
%load_ext lab_black

  Preparing metadata (setup.py) ... done
  Created wheel for nb_black: filename=nb_black-1.0.7-py3-none-any.whl size=5297 sha256=abc1a8828b335b2d7dd908ed7f408d691082934e55bf7fb67627c5fbbd683a02
  Stored in directory: /root/.cache/pip/wheels/1e/b2/88/51c66d23ea5fd0d40ed50997555e15d981d92671376a9a412a
Successfully built nb_black


In [3]:
MAIN_PATH = "/kaggle/input/lg-18650hg2-liion-battery-data/Dataset_Li-ion/LG_HG2_Original_Dataset_McMasterUniversity_Jan_2020"

A brand new 3Ah LG HG2 cell was tested in an 8 cu.ft. thermal chamber with a 75amp, 5 volt Digatron Firing Circuits Universal Battery Tester channel with a voltage and current accuracy of 0.1% of full scale. these data are used in the design process of an SOC estimator using a deep feedforward neural network (FNN) approach. The data also includes a description of data acquisition, data preparation, development of an FNN example script.

A new 3Ah LG HG2 cell was tested in an 8 cu.ft. thermal chamber with a 75amp, 5 volt Digatron Firing Circuits Universal Battery Tester channel with a voltage and current accuracy of 0.1% of full scale.

In [7]:
train1 = os.path.join(MAIN_PATH, "25degC")
l = os.listdir(train1)
l = [i for i in l if i.endswith(".csv")]
l

['549_Dis_2C.csv',
 '551_Charge5.csv',
 '551_Charge6.csv',
 '551_Charge2.csv',
 '551_Charge1.csv',
 '552_Charge12.csv',
 '549_HPPC.csv',
 '552_Charge9.csv',
 '551_LA92.csv',
 '552_Mixed6.csv',
 '552_Charge11.csv',
 '551_HWFET.csv',
 '552_Mixed8.csv',
 '551_Charge3.csv',
 '552_Charge15.csv',
 '551_Mixed1.csv',
 '551_Charge8.csv',
 '551_UDDS.csv',
 '551_Charge4.csv',
 '552_Cap_1C.csv',
 '552_Charge13.csv',
 '552_Mixed7.csv',
 '552_Charge16.csv',
 '552_Mixed4.csv',
 '549_Dis_0p5C.csv',
 '549_C20DisCh.csv',
 '551_Charge7.csv',
 '551_US06.csv',
 '551_Mixed2.csv',
 '549_Charge.csv',
 '552_Charge10.csv',
 '552_Mixed5.csv',
 '552_PausCycl.csv',
 '552_Mixed3.csv',
 '552_Charge14.csv',
 '551_Cap_1C.csv']

In [8]:
train1_csv = os.path.join(train1, "551_LA92.csv")
train1_csv

'/kaggle/input/lg-18650hg2-liion-battery-data/Dataset_Li-ion/LG_HG2_Original_Dataset_McMasterUniversity_Jan_2020/25degC/551_LA92.csv'

In [9]:
df_train1_csv = pd.read_csv(train1_csv, skiprows=30)
df_train1_csv.head()

,10/29/2018 3:53:42 AM,39,TABLE,16:50:37.323,00:00:01.673,1,1.1,LG_HG2_CyclesA,4.19155,-0.05108,23.76583,-0.00000,-0.00000.1,5.00000,Unnamed: 14
0,10/29/2018 3:53:42 AM,39,TABLE,16:50:37.423,00:00:01.773,1,1,LG_HG2_CyclesA,4.19088,-0.08173,23.76583,-0.00000,-0.00001,5.0,NaN
1,10/29/2018 3:53:42 AM,39,TABLE,16:50:37.522,00:00:01.872,1,1,LG_HG2_CyclesA,4.19054,-0.08939,23.76583,-0.00000,-0.00002,5.0,NaN
2,10/29/2018 3:53:42 AM,39,TABLE,16:50:37.622,00:00:01.972,1,1,LG_HG2_CyclesA,4.19037,-0.09195,23.76583,-0.00001,-0.00003,5.0,NaN
3,10/29/2018 3:53:42 AM,39,TABLE,16:50:37.723,00:00:02.073,1,1,LG_HG2_CyclesA,4.19037,-0.09195,23.76583,-0.00001,-0.00004,5.0,NaN
4,10/29/2018 3:53:42 AM,39,TABLE,16:50:37.821,00:00:02.171,1,1,LG_HG2_CyclesA,4.19037,-0.09195,23.76583,-0.00001,-0.00005,5.0,NaN


In [10]:
df_train1_csv.columns = [
    "Time Stamp",
    "Step",
    "Status",
    "Prog Time",
    "Step Time",
    "Cycle",
    "Cycle Level",
    "Procedure",
    "Voltage",
    "Current",
    "Temperature",
    "Capacity",
    "WhAccu",
    "Cnt",
    "Empty",
]

In [10]:
df_train1_csv = df_train1_csv.iloc[:, :-1]  # run only once
df_train1_csv.head(3)

,Time Stamp,Step,Status,Prog Time,Step Time,Cycle,Cycle Level,Procedure,Voltage,Current,Temperature,Capacity,WhAccu,Cnt
0,10/29/2018 3:53:42 AM,39,TABLE,16:50:37.423,00:00:01.773,1,1,LG_HG2_CyclesA,4.19088,-0.08173,23.76583,-0.00000,-0.00001,5.0
1,10/29/2018 3:53:42 AM,39,TABLE,16:50:37.522,00:00:01.872,1,1,LG_HG2_CyclesA,4.19054,-0.08939,23.76583,-0.00000,-0.00002,5.0
2,10/29/2018 3:53:42 AM,39,TABLE,16:50:37.622,00:00:01.972,1,1,LG_HG2_CyclesA,4.19037,-0.09195,23.76583,-0.00001,-0.00003,5.0


In [11]:
df_train1_csv.describe()

,Step,Cycle,Cycle Level,Voltage,Current,Temperature,Capacity,WhAccu,Cnt
count,100763.000000,100763.000000,100763.000000,100763.000000,100763.000000,100763.000000,100763.000000,100763.000000,100763.0
mean,39.119111,3.590197,0.940444,3.702031,-0.926346,24.246026,-1.336233,-4.964102,5.0
std,0.473326,2.064147,0.236663,0.282690,2.406846,0.222317,0.789170,2.811615,0.0
min,39.000000,0.000000,0.000000,2.859260,-12.144700,23.660670,-2.590260,-9.263270,5.0
25%,39.000000,2.000000,1.000000,3.499510,-2.331880,24.081300,-1.999075,-7.365255,5.0
50%,39.000000,4.000000,1.000000,3.716890,-0.155800,24.186460,-1.303810,-4.958720,5.0
75%,39.000000,5.000000,1.000000,3.946430,0.000000,24.396780,-0.663455,-2.591270,5.0
max,41.000000,7.000000,1.000000,4.208720,5.999610,25.027730,-0.000000,-0.000010,5.0


In [12]:
df_train1_csv = df_train1_csv[
    (df_train1_csv["Status"] == "TABLE") | (df_train1_csv["Status"] == "DCH")
]
df_train1_csv.head(3)

,Time Stamp,Step,Status,Prog Time,Step Time,Cycle,Cycle Level,Procedure,Voltage,Current,Temperature,Capacity,WhAccu,Cnt
0,10/29/2018 3:53:42 AM,39,TABLE,16:50:37.423,00:00:01.773,1,1,LG_HG2_CyclesA,4.19088,-0.08173,23.76583,-0.00000,-0.00001,5.0
1,10/29/2018 3:53:42 AM,39,TABLE,16:50:37.522,00:00:01.872,1,1,LG_HG2_CyclesA,4.19054,-0.08939,23.76583,-0.00000,-0.00002,5.0
2,10/29/2018 3:53:42 AM,39,TABLE,16:50:37.622,00:00:01.972,1,1,LG_HG2_CyclesA,4.19037,-0.09195,23.76583,-0.00001,-0.00003,5.0


In [13]:
max_discharge = abs(min(df_train1_csv["Capacity"]))  # get maximum discharge
df_train1_csv["SoC Capacity"] = max_discharge + df_train1_csv["Capacity"]
df_train1_csv["SoC Percentage"] = df_train1_csv["SoC Capacity"] / max(
    df_train1_csv["SoC Capacity"]
)

In [ ]:
df_train1_csv.shape

In [ ]:
df_train1_csv.info()

## EDA

In [ ]:
df_train1_csv["Prog Time"]

In [ ]:
# time_parts = input_string.split(':')
#         second_parts = time_parts[2].split('.')
#         return timedelta(hours=int(time_parts[0]),
#             minutes=int(time_parts[1]),
#             seconds=int(second_parts[0]),
#             microseconds=int(second_parts[1])).total_seconds()
t1 = df_train1_csv["Prog Time"][0]
time_parts = t1.split(":")
second_part = time_parts[2].split(".")
timedelta(
    hours=int(time_parts[0]),
    minutes=int(time_parts[1]),
    seconds=int(second_part[0]),
    microseconds=int(second_part[1]),
).total_seconds()

In [ ]:
df_train1_csv.isnull().sum()

SOC vs Voltage

In [ ]:
# %%time
# import plotly.graph_objects as go

# fig = go.Figure()

# fig.add_trace(go.Scatter(x = df_train1_csv['Voltage'], y = df_train1_csv['SoC Percentage'],
#                         mode='lines', name='SOC Vs Train'))

# fig.update_layout(title='SOC Vs Train',
#                   xaxis_title='Voltage',
#                   yaxis_title='SOC%')

# fig.show()

In [ ]:
# # df_train1_csv
# x_val = df_train1_csv["Voltage"].to_numpy()
# y_val = df_train1_csv["SoC Percentage"].to_numpy()
# plt.figure(figsize=(20, 5))
# sns.lineplot(y=y_val, x=x_val, data=df_train1_csv, color="orange")
# plt.xlabel("Voltage")
# plt.ylabel("SOC %")
# plt.title(
#     "SOC Vs Voltage",
#     fontdict={
#         "fontsize": 20,
#         "fontweight": 3,
#         "verticalalignment": "center",
#         "horizontalalignment": "center",
#     },
# )
# plt.show()

SOC overtime

In [ ]:
# %%time
# import plotly.graph_objects as go

# fig = go.Figure()

# fig.add_trace(go.Scatter(x = df_train1_csv['Time Stamp'], y = df_train1_csv['SoC Percentage'],
#                         mode='lines', name='SOC Vs Time'))

# fig.update_layout(title='SOC Vs Time',
#                   xaxis_title='Time',
#                   yaxis_title='SOC%')

# fig.show()

In [ ]:
# %%time
# x_val = df_train1_csv['Time Stamp'].to_numpy()
# y_val = df_train1_csv['SoC Percentage'].to_numpy()

# x_ticks_val = np.arange(0, 94762, 94762) # values to display on the x axis

# plt.figure(figsize=(20, 5))
# sns.lineplot(y = y_val, x = x_val, data = df_train1_csv, color='blue')
# plt.xlabel('Time')
# plt.ylabel('SOC %')
# plt.title('SOC Vs Time', 
#           fontdict={'fontsize': 20,
#         'fontweight': 3, 'verticalalignment': 'center',
#         'horizontalalignment': 'center'})
# plt.xticks([])
# plt.show()

In [ ]:
# plt.figure(figsize=(20, 5))
# a = np.arange(100)
# # 94762/ 7
# sns.lineplot(y = y_val[a], x = x_val[a], data = df_train1_csv.iloc[a], color='blue')
# plt.xticks([])

# for i in a[0::10]:
#     plt.axvline(i, ls='--')

# plt.show()

In [ ]:
# %%time
# fig = make_subplots(rows=1, cols=3)

# fig.add_trace(
#     go.Scatter(x=df_train1_csv["Voltage"], y=df_train1_csv["SoC Percentage"], mode="markers",name="Scatter 1"),
#     row=1, col=1
# )

# fig.add_trace(
#     go.Scatter(x=df_train1_csv["Current"],  y=df_train1_csv["SoC Percentage"], mode="markers",name="Scatter 2"),
#     row=1, col=2
# )

# fig.add_trace(
#     go.Scatter(x=df_train1_csv["Temperature"],  y=df_train1_csv["SoC Percentage"], mode="markers", name="Scatter 3",),
#     row=1, col=3
# )

# fig.show()
# # plt.figure(figsize=(20, 5))
# # sns.pairplot(data=df_train1_csv, x_vars = ['Voltage','Current','Temperature', 'Capacity'], 
# #             y_vars = ['SoC Percentage'])
# # plt.show()

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(18, 5))
sns.kdeplot(data=df_train1_csv, x="Voltage", ax=axes[0])
sns.kdeplot(data=df_train1_csv, x="Current", ax=axes[1])
plt.show()

In [ ]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler

sc = StandardScaler()

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(18, 5))

x = sc.fit_transform(df_train1_csv["Voltage"].to_numpy().reshape(-1, 1))
x1 = sc.fit_transform(df_train1_csv["Current"].to_numpy().reshape(-1, 1))
sns.kdeplot(data=x, ax=axes[0])
sns.kdeplot(data=x1, ax=axes[1])
plt.show()

Separate X and Y values

In [ ]:
df_train1_csv[["Voltage", "Current", "Temperature"]].head()

In [ ]:
x = df_train1_csv[["Voltage", "Current", "Temperature"]].to_numpy()
y = df_train1_csv[["SoC Percentage"]].to_numpy()

In [ ]:
cycles = []
cycles.append((x, y))

In [ ]:
# cycles[0] # x, y pair
# print(cycles[0][0])) # gives x
# print(cycles[0][1]) # gives Y
# print(cycles[0][0][0]) # gives voltage, current, temperature values

In [ ]:
# np.array(cycles[0][0:0 + steps])#.reshape(1, 300, x_tem_length)

In [ ]:
steps = 300
x_length = len(cycles[0][0][0])
y_length = len(cycles[0][1][0])
x = np.empty((0, steps, x_length), float)
y = np.empty((0, steps, y_length), float)
# for cycle in cycles:
#     for i in range(0, len(cycle[0]) - steps, steps):
#         next_x = np.array(cycle[0][i:i + steps]).reshape(1, steps, x_length)
#         next_y = np.array(cycle[1][i:i + steps]).reshape(1, steps, y_length)
#         x = np.concatenate((x, next_x))
#         y = np.concatenate((y, next_y))

In [ ]:
# next_x = np.array(cycle)
x_new = np.concatenate((x, np.array(cycles[0][0][0:300]).reshape(1, 300, x_length)))

In [ ]:
x_new[:, 0:5]

## Step 1 Data Cleaning

**Function to gat data from the train and test data**<br>
First we will only focus in data that is recorded ranging from 0 to 25 degree celcius.

In [12]:
MAIN_PATH = "/kaggle/input/lg-18650hg2-liion-battery-data/Dataset_Li-ion/LG_HG2_Original_Dataset_McMasterUniversity_Jan_2020/"

In [65]:
def get_discharge_whole_cycle(
    train_files, test_files, output_capacity=False, scale_test=False
):
    """
    This function returns the train and test data for all the cycle
    according to the file paths provided
    and cales according to the user input.
    """
    train = get_data(train_files, output_capacity)
    test = get_data(test_files, output_capacity)
    if scale_test:
        train, test, scaling_factors = scale_x(train, test, scale_test=scale_test)
        return (train, test), scaling_factors
    else:
        train, test = scale_x(train, test, scale_test=scale_test)
        return (train, test)


# --------------------------------------------------------------------------#


def get_data(names, output_capacity):
    """
    Returns a tuple of (x, y) values for every file path
    """
    cycles = []  # list to store tuple
    for name in names:
        print(f"Extracting for {name}...")
        cycle = pd.read_csv(MAIN_PATH + name + ".csv", skiprows=30)
        cycle.columns = [
            "Time Stamp",
            "Step",
            "Status",
            "Prog Time",
            "Step Time",
            "Cycle",
            "Cycle Level",
            "Procedure",
            "Voltage",
            "Current",
            "Temperature",
            "Capacity",
            "WhAccu",
            "Cnt",
            "Empty",
        ]

        cycle = cycle[(cycle["Status"] == "TABLE") | (cycle["Status"] == "DCH")]
        max_discharge = abs(min(cycle["Capacity"]))  # get maximum capacity
        cycle["SoC Capacity"] = max_discharge + cycle["Capacity"]  # soc capacity
        cycle["SoC Percentage"] = cycle["SoC Capacity"] / max(
            cycle["SoC Capacity"]
        )  # compute soc for each timestep

        x = cycle[["Voltage", "Current", "Temperature"]].to_numpy()

        if output_capacity:  # if we want output capacity
            y = cycle[["SoC Capacity"]].to_numpy()
        else:
            y = cycle[["SoC Percentage"]].to_numpy()

        if np.isnan(np.min(x)) or np.isnan(
            np.min(y)
        ):  # if there is any nan value in the dataset
            print(f"Nan value at cycle :{name}, removing the nan...")
            x = [~np.isnan(x).any(axis=1)]
            y = [~np.isnan(y).any(axis=1)].reshape(-1, y.shape[1])

        cycles.append((x, y))
    return cycles


# ---------------------------------------------------------------#


def scale_x(train, test, scale_test=False):
    """
    Use min-max feature scaling to scale down the features on a uniform scale.
    """
    scaling_factors = []  # save scaling factors for each feature
    for index in range(len(train[0][0][0])):
        f_min = min([min(cycle[0][0:, index]) for cycle in train])
        f_max = max([max(cycle[0][0:, index]) for cycle in train])
        scaling_factors.append((f_min, f_max))  # save scaling factors for this feature

        for i in range(len(train)):  # use min max scaling
            train[i][0][:index] = (train[i][0][:index] - f_min) / (f_max - f_min)

        if scale_test:
            for i in range(len(test)):
                test[i][0][:, index] = (test[i][0][:, index] - f_min) / (f_max - f_min)
    if scale_test:
        return train, test, scaling_factors
    else:
        return train, scaling_factors


# ---------------------------------------------------------------#

**Function to split the dataset into train and test**

In [59]:
def get_discharge_multiple_step(cycles, steps=300):
    train_x, train_y = _split_to_multiple_step(cycles[0], steps)
    test_x, test_y = _split_to_multiple_step(cycles[1], steps)
    return (train_x, train_y, test_x, test_y)


# ---------------------------------------------------------------#


def _split_to_multiple_step(cycles, steps):
    x_length = len(cycles[0][0][0])
    y_length = len(cycles[0][1][0])
    x = np.empty((0, steps, x_length), float)
    y = np.empty((0, steps, y_length), float)
    for cycle in cycles:
        for i in range(0, len(cycle[0]) - steps, steps):
            next_x = np.array(cycle[0][i : i + steps]).reshape(1, steps, x_length)
            next_y = np.array(cycle[1][i : i + steps]).reshape(1, steps, y_length)
            x = np.concatenate((x, next_x))
            y = np.concatenate((y, next_y))
        return x, y


# ---------------------------------------------------------------#


def keep_only_y_end(y, step, is_stateful=False):
    if is_stateful:
        return y[:, :, ::step]
    else:
        return y[:, ::step]

**Set up training data, test data and other parameters**

In [60]:
train_files = [
    "0degC/589_Mixed1",
    "0degC/589_Mixed2",
    "0degC/590_Mixed4",
    "0degC/590_Mixed5",
    "0degC/590_Mixed6",
    "0degC/590_Mixed8",
    "10degC/567_Mixed1",
    "10degC/567_Mixed2",
    "10degC/571_Mixed4",
    "10degC/571_Mixed5",
    "10degC/571_Mixed6",
    "10degC/571_Mixed8",
    "25degC/551_Mixed1",
    "25degC/551_Mixed2",
    #'25degC/552_Mixed3',
    "25degC/552_Mixed4",
    "25degC/552_Mixed5",
    "25degC/552_Mixed6",
    "25degC/552_Mixed8",
]
test_files = [
    "0degC/589_LA92",
    "0degC/589_UDDS",
    "0degC/589_US06",
    "0degC/590_Mixed7",
    #     "10degC/582_LA92",
    "10degC/576_UDDS",
    "10degC/567_US06",
    "10degC/571_Mixed7",
    "25degC/551_LA92",
    "25degC/551_UDDS",
    "25degC/551_US06",
    "25degC/552_Mixed7",
]
steps = 300
# stateful_config = True

In [66]:
%%time
cycles, scaling_factors = get_discharge_whole_cycle(train_files, test_files, output_capacity=False, scale_test=True)

Extracting for 0degC/589_Mixed1...
Extracting for 0degC/589_Mixed2...
Extracting for 0degC/590_Mixed4...
Extracting for 0degC/590_Mixed5...
Extracting for 0degC/590_Mixed6...
Extracting for 0degC/590_Mixed8...
Extracting for 10degC/567_Mixed1...
Extracting for 10degC/567_Mixed2...
Extracting for 10degC/571_Mixed4...
Extracting for 10degC/571_Mixed5...
Extracting for 10degC/571_Mixed6...
Extracting for 10degC/571_Mixed8...
Extracting for 25degC/551_Mixed1...
Extracting for 25degC/551_Mixed2...
Extracting for 25degC/552_Mixed4...
Extracting for 25degC/552_Mixed5...
Extracting for 25degC/552_Mixed6...
Extracting for 25degC/552_Mixed8...
Extracting for 0degC/589_LA92...
Extracting for 0degC/589_UDDS...
Extracting for 0degC/589_US06...
Extracting for 0degC/590_Mixed7...
Extracting for 10degC/576_UDDS...
Extracting for 10degC/567_US06...
Extracting for 10degC/571_Mixed7...
Extracting for 25degC/551_LA92...
Extracting for 25degC/551_UDDS...
Extracting for 25degC/551_US06...
Extracting for 25d

In [68]:
print(
    f"First element of cycles has {len(cycles[0])} files and they belong to training data.\nSecond element of cycles has {len(cycles[1])} files and they belong to test data."
)

print("Train/Test %d" % len(cycles))
print("x/y :%d" % len(cycles[0][0]))
print("time_steps: %d" % len(cycles[0][0][0]))
print("x features: %d" % len(cycles[0][0][0][0]))

First element of cycles has 18 files and they belong to training data.
Second element of cycles has 11 files and they belong to test data.
Train/Test 2
x/y :2
time_steps: 61310
x features: 3


In [18]:
# len(cycles[0][0][0])  # time_steps: 61310
# max(len(cycle[0]) for cycle in cycles[0])  # 74848
# max(len(cycle[0]) for cycle in cycles[1])  # 142896

In [27]:
cycles[0][0][0][0]

array([0.06508161, 0.05856657, 0.05755807])

In [78]:
def get_stateful_cycle(cycles, pad_num=0, steps=100):
    max_length = max(
        max(len(cycle[0]) for cycle in cycles[0]),
        max(len(cycle[0]) for cycle in cycles[1]),
    )
    train_x, train_y = _to_padded_cycle(cycles[0], pad_num, max_length)
    test_x, test_y = _to_padded_cycle(cycles[1], pad_num, max_length)
    train_x = _split_cycle(train_x, steps)
    train_y = _split_cycle(train_y, steps)
    test_x = _split_cycle(test_x, steps)
    test_y = _split_cycle(test_y, steps)
    return (train_x, train_y, test_x, test_y)


# ---------------------------------------------------------------#


def _to_padded_cycle(cycles, pad_num, max_length):
    x_length = len(cycles[0][0][0])
    y_length = len(cycles[0][1][0])
    x = np.full((len(cycles), max_length, x_length), pad_num, dtype=float)
    y = np.full((len(cycles), max_length, y_length), pad_num, dtype=float)
    for i, cycle in enumerate(cycles):
        x[i, : cycle[0].shape[0]] = cycle[0]
        y[i, : cycle[1].shape[0]] = cycle[1]
    return x, y


# ---------------------------------------------------------------#


def _split_cycle(cycles, steps):
    features = cycles.shape[2]
    time_steps = cycles.shape[1]
    new_cycles = np.empty((0, time_steps // steps, steps, features), float)
    for cycle in cycles:
        new_cycle = np.empty((0, steps, features), float)
        for i in range(0, len(cycle) - steps, steps):
            next_split = np.array(cycle[i : i + steps]).reshape(1, steps, features)
            new_cycle = np.concatenate((new_cycle, next_split))
        new_cycles = np.concatenate(
            (new_cycles, new_cycle.reshape(1, time_steps // steps, steps, features))
        )
    return new_cycles

Split the data into train and test

In [28]:
# new
train_x, train_y, test_x, test_y = get_discharge_multiple_step(cycles, steps)

In [ ]:
# train_x, train_y, test_x, test_y = get_discharge_multiple_step(cycles, 300)

Check the dimentions and visualize the training and test data

In [29]:
train_x.shape, train_y.shape, test_x.shape, test_y.shape

((204, 300, 3), (204, 300, 1), (251, 300, 3), (251, 300, 1))

In [30]:
train_x[0][0][:5]  # training x

array([0.06508161, 0.05856657, 0.05755807])

In [31]:
test_x[0][:5]

array([[0.96146405, 0.74758744, 0.00387602],
       [0.96053123, 0.74716384, 0.00387602],
       [0.95971587, 0.74695183, 0.00387602],
       [0.95913545, 0.74695183, 0.00387602],
       [0.95867249, 0.74695183, 0.00775205]])

In [32]:
train_y[:5]  # training y

array([[[1.        ],
        [0.99999574],
        [0.99999574],
        ...,
        [0.99354825],
        [0.99353548],
        [0.99352697]],

       [[0.99351846],
        [0.9935142 ],
        [0.99350995],
        ...,
        [0.99205022],
        [0.99205447],
        [0.99206299]],

       [[0.9920715 ],
        [0.99207575],
        [0.99208426],
        ...,
        [0.98984147],
        [0.98982019],
        [0.98980317]],

       [[0.98978615],
        [0.98976487],
        [0.98974785],
        ...,
        [0.98923715],
        [0.9892329 ],
        [0.9892329 ]],

       [[0.9892329 ],
        [0.9892329 ],
        [0.98922864],
        ...,
        [0.98741143],
        [0.98740717],
        [0.98740717]]])

In [33]:
test_y[:5]  # testing y

array([[[1.        ],
        [1.        ],
        [0.99999574],
        ...,
        [0.99966381],
        [0.99965956],
        [0.99965956]],

       [[0.99965956],
        [0.9996553 ],
        [0.9996553 ],
        ...,
        [0.99837864],
        [0.99837864],
        [0.99837864]],

       [[0.99837864],
        [0.99837864],
        [0.99837864],
        ...,
        [0.99558275],
        [0.99554871],
        [0.99551466]],

       [[0.99547636],
        [0.99543806],
        [0.99539551],
        ...,
        [0.99368053],
        [0.99367627],
        [0.99367627]],

       [[0.99367202],
        [0.99366776],
        [0.99365925],
        ...,
        [0.98988885],
        [0.98988459],
        [0.98988459]]])

In [34]:
train_y = keep_only_y_end(train_y, steps)
test_y = keep_only_y_end(test_y, steps)

In [56]:
# display_x = train_x.reshape(
#     train_x.shape[0] * train_x.shape[1] * train_x.shape[2], train_x.shape[3]
# )
# display_y = train_y.reshape(
#     train_y.shape[0] * train_y.shape[1] * train_y.shape[2], train_y.shape[3]
# )

# display_y

Display the X values

In [ ]:
# from plotly.subplots import make_subplots

# fig = go.Figure()

# fig = plotly.subplots.make_subplots(rows=3, cols=1, shared_xaxes=True)

# fig.add_trace(go.Scatter(y=display_x[:,0], mode='lines', name='Voltage'), row=1, col=1)
# fig.update_yaxes(title_text="Voltage", row=1, col=1)

# fig.add_trace(go.Scatter(y=display_x[:,1], mode='lines', name='Current'), row=2, col = 1)
# fig.update_yaxes(title_text="Current", row=2, col=1)

# fig.add_trace(go.Scatter(y=display_x[:,2],mode='lines', name='Temperature'), row = 3, col = 1)
# fig.update_yaxes(title_text="Temperature", row=3, col=1)

# fig.show()

Display the y values

In [ ]:
# fig = go.Figure()
# fig.add_trace(go.Scatter(y=display_y[:,0],
#                         mode='lines', name='SoC'))
# fig.update_layout(title='Y Data',
#                     xaxis_title='Step',
#                     yaxis_title='SoC',
#                     width=1000,
#                     height=500)
# fig.show()

## Training the Model

In [35]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from tensorflow.keras.optimizers import SGD, Adam
from keras.utils import np_utils
from keras.layers import LSTM, Embedding, RepeatVector, TimeDistributed, Masking
from keras.callbacks import EarlyStopping, ModelCheckpoint, LambdaCallback

## Iter 2

In [ ]:
# def build_model(batch_size):
#     opt = tf.keras.optimizers.Adam(learning_rate=0.00001)
#     model = Sequential()
#     model.add(LSTM(512, activation='tanh',
#                     return_sequences=True,
#                     stateful=True,batch_size=batch_size,
#                     input_shape=(train_x.shape[2], train_x.shape[3])))
#     model.add(LSTM(512, activation='tanh', return_sequences=True, stateful=True))
#     model.add(LSTM(512, activation='tanh', return_sequences=True, stateful=True))
#     model.add(Dense(256, activation='selu'))
#     model.add(Dense(128, activation='selu'))
#     model.add(Dense(1, activation='linear'))
#     model.summary()
#     model.compile(optimizer=opt, loss='huber', metrics=['mse', tf.keras.metrics.RootMeanSquaredError(name='rmse')])
#     return model

In [ ]:
# EXPERIMENT = "lstm_soc_lg_stateful"

# itc = pytz.timezone('Asia/Kolkata')

# experiment_name = datetime.now(itc).strftime('%Y:%m:%d %H:%M:%S') + '_' + EXPERIMENT

# print(experiment_name)

# model = build_model(train_x.shape[0])

In [ ]:
# %%time

# epochs = 200

# hist = {'loss':[], 'mse':[], 'rmse':[]}
# for epoch in range(epochs):
#     avg_hist = {'loss':[], 'mse':[], 'rmse':[]}
#     shuffled_cycle_index = random.sample(range(train_x.shape[0]), train_x.shape[0]) # select chunks from the 18 chunks created
#     for chunk in range(train_x.shape[1]):
#         batch_x = []
#         batch_y = []
#         for i in shuffled_cycle_index:
#             batch_x.append(train_x[i, chunk, :, :]) # choose a random batch for given chunk and i
#             batch_y.append(train_y[i, chunk, :, :])
#         batch_x = np.array(batch_x)
#         batch_y = np.array(batch_y)
#         results = model.fit(batch_x, batch_y, shuffle=False, verbose=0, batch_size=train_x.shape[0])
#         avg_hist["loss"].append(results.history["loss"]) # append all metrics to the dictionary
#         avg_hist["mse"].append(results.history["mse"])
#         avg_hist["rmse"].append(results.history["rmse"])
#         print('\rChunk {}/{} - loss: {:.4e} - mse: {:.4e} - rmse: {}'.format(chunk, train_x.shape[1], np.mean(avg_hist["loss"]), np.mean(avg_hist["mse"]), np.mean(avg_hist["rmse"])), end="")
#     hist["loss"].append(np.mean(avg_hist["loss"]))
#     hist["mse"].append(np.mean(avg_hist["mse"]))
#     hist["rmse"].append(np.mean(avg_hist["rmse"]))
#     print("\rEpoch {}/{} - loss: {:.4e} - mse: {:.4e} - rmse: {}".format(epoch+1, epochs, hist["loss"][-1], hist["mse"][-1], hist["rmse"][-1]))
#     model.reset_states()

In [ ]:
# model.save(data_path + 'results/trained_model/%s.h5' % experiment_name)

# hist_df = pd.DataFrame(history)
# hist_csv_file = data_path + 'results/trained_model/%s_history.csv' % experiment_name
# with open(hist_csv_file, mode='w') as f:
#     hist_df.to_csv(f)

## Iter 1 300 steps

In [98]:
itc = pytz.timezone("Asia/Kolkata")

print("UTC in Default Format : ", datetime.now(itc).strftime("%Y:%m:%d-%H:%M:%S"))

UTC in Default Format :  2023:04:19-16:38:01


In [99]:
EXPERIMENT = "lstm_soc_percentage_lg_positive_temp_300_steps_mixed_cycle_test"

experiment_name = datetime.now(itc).strftime("%Y:%m:%d-%H:%M:%S") + "_" + EXPERIMENT
print(f"We are performing : {experiment_name}")

opt = tf.keras.optimizers.Adam(learning_rate=0.00001)

model = Sequential()
model.add(
    LSTM(
        256,
        activation="selu",
        return_sequences=True,
        input_shape=(train_x.shape[1], train_x.shape[2]),
    )
)
model.add(LSTM(256, activation="selu", return_sequences=False))
model.add(Dense(256, activation="selu"))
model.add(Dense(128, activation="selu"))
model.add(Dense(1, activation="linear"))
model.summary()

We are performing : 2023:04:19-16:38:03_lstm_soc_percentage_lg_positive_temp_300_steps_mixed_cycle_test
Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_4 (LSTM)                (None, 300, 256)          266240    
_________________________________________________________________
lstm_5 (LSTM)                (None, 256)               525312    
_________________________________________________________________
dense_6 (Dense)              (None, 256)               65792     
_________________________________________________________________
dense_7 (Dense)              (None, 128)               32896     
_________________________________________________________________
dense_8 (Dense)              (None, 1)                 129       
Total params: 890,369
Trainable params: 890,369
Non-trainable params: 0
_________________________________________________________________


In [100]:
model.compile(
    optimizer=opt,
    loss="huber",
    metrics=["mse", "mae", "mape", tf.keras.metrics.RootMeanSquaredError(name="rmse")],
)

mc = ModelCheckpoint(
    "/kaggle/working/" + "results/trained_model/%s_best.h5" % experiment_name,
    save_best_only=True,
    monitor="val_loss",
)

In [101]:
%%time
history = model.fit(train_x, train_y, 
                                epochs=500, 
                                batch_size=32, 
                                verbose=1,
                                validation_split=0.2,
                                callbacks = [mc]
                               )

Epoch 1/500
6/6 [==============================] - 10s 1s/step - loss: 0.0939 - mse: 0.1893 - mae: 0.3247 - mape: 60.3242 - rmse: 0.4351 - val_loss: 0.0961 - val_mse: 0.1921 - val_mae: 0.3909 - val_mape: 259.7776 - val_rmse: 0.4383
Epoch 2/500
6/6 [==============================] - 8s 1s/step - loss: 0.0435 - mse: 0.0870 - mae: 0.2344 - mape: 52.3882 - rmse: 0.2950 - val_loss: 0.0338 - val_mse: 0.0677 - val_mae: 0.2191 - val_mape: 183.7051 - val_rmse: 0.2602
Epoch 3/500
6/6 [==============================] - 8s 1s/step - loss: 0.0372 - mse: 0.0743 - mae: 0.2367 - mape: 42.5514 - rmse: 0.2726 - val_loss: 0.0393 - val_mse: 0.0787 - val_mae: 0.2481 - val_mape: 183.7891 - val_rmse: 0.2805
Epoch 4/500
6/6 [==============================] - 8s 1s/step - loss: 0.0262 - mse: 0.0524 - mae: 0.2030 - mape: 39.5711 - rmse: 0.2290 - val_loss: 0.0731 - val_mse: 0.1463 - val_mae: 0.3621 - val_mape: 276.1231 - val_rmse: 0.3824
Epoch 5/500
6/6 [==============================] - 8s 1s/step - loss: 0.023

In [102]:
model.save("/kaggle/working/" + "results/trained_model/%s.h5" % experiment_name)

history_df = pd.DataFrame(history.history)
history_csv_file = (
    "/kaggle/working/" + "results/trained_model/%s_history.csv" % experiment_name
)
with open(history_csv_file, mode="w") as f:
    history_df.to_csv(f)

## Test the model

In [103]:
model.evaluate(test_x, test_y)

8/8 [==============================] - 1s 93ms/step - loss: 0.8474 - mse: 1.9490 - mae: 1.3429 - mape: 429.0428 - rmse: 1.3961


[0.8474173545837402,
 1.9490442276000977,
 1.342926025390625,
 429.04278564453125,
 1.3960816860198975]

In [ ]:
# iter 2
# model.evaluate(test_x, test_y)

## Visualize the result

In [ ]:
# from datetime import date
# date = date.today().strftime("%b-%d-%Y")
# date

In [ ]:
# os.listdir('/kaggle/working/results/trained_model')

In [ ]:
# # file = 
# result_df = pd.read_csv('/kaggle/working/results/trained_model/2023:03:01 00:39:37_lstm_soc_percentage_lg_positive_temp_300_steps_mixed_cycle_test_history.csv')
# result_df.head()

In [ ]:
# result_df.to_csv('iter3.csv')

### Plotting trainining and Validation loss

In [104]:
fig = go.Figure()
fig.add_trace(go.Scatter(y=history.history["loss"], mode="lines", name="train"))
fig.add_trace(
    go.Scatter(y=history.history["val_loss"], mode="lines", name="validation")
)
fig.update_layout(title="Loss trend", xaxis_title="epoch", yaxis_title="loss")
fig.show()

In [ ]:
# # iter 2
# fig = go.Figure()
# fig.add_trace(go.Scatter(y = history.history['loss'], mode='lines', name = 'train'))

# fig.add_trace(go.Scatter(y = history.history['val_loss'], mode='lines', name = 'validation'))

# fig.update_layout(title='Loss trend',
#                   xaxis_title='epoch',
#                   yaxis_title='loss')
# fig.show()

### Predictions on training

In [106]:
train_predictions = model.predict(train_x)

In [107]:
cycle_num = 0
steps_num = 8000
step_index = np.arange(cycle_num * steps_num, (cycle_num + 1) * steps_num)

fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=step_index,
        y=train_predictions.flatten()[
            cycle_num * steps_num : (cycle_num + 1) * steps_num
        ],
        mode="lines",
        name="SoC predicted",
    )
)
fig.add_trace(
    go.Scatter(
        x=step_index,
        y=train_y.flatten()[cycle_num * steps_num : (cycle_num + 1) * steps_num],
        mode="lines",
        name="SoC actual",
    )
)
fig.update_layout(
    title="Results on training", xaxis_title="Step", yaxis_title="SoC percentage"
)
fig.show()

### Predictions on testing

In [78]:
def inverse_transform(test, scaling_factors):
    """
    Inverse transform a scaled test set using the scaling factors.
    """
    for index in range(len(scaling_factors)):
        f_min, f_max = scaling_factors[index]
        print(test[:, index])
        test[:, index] = (test[:, index] * (f_max - f_min)) + f_min
    return test

In [108]:
test_predictions = model.predict(test_x)

In [90]:
# for i in range(len(predictions)):
#     test_predictions[i] = inverse_transform(test_predictions[i], scaling_factors)

In [109]:
cycle_num = 0
steps_num = 8000
step_index = np.arange(cycle_num * steps_num, (cycle_num + 1) * steps_num)

fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=step_index,
        y=test_predictions.flatten()[
            cycle_num * steps_num : (cycle_num + 1) * steps_num
        ],
        mode="lines",
        name="SoC predicted",
    )
)
fig.add_trace(
    go.Scatter(
        x=step_index,
        y=test_y.flatten()[cycle_num * steps_num : (cycle_num + 1) * steps_num],
        mode="lines",
        name="SoC actual",
    )
)
fig.update_layout(
    title="Results on testing", xaxis_title="Step", yaxis_title="SoC percentage"
)
fig.show()

In [97]:
# test_y.flatten()
# test_predictions